In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

def load_dataset():
    # opening the training set .h5 file in read mode
    train_dataset = h5py.File("train_catvnoncat.h5", "r")
    #extract the training images
    train_set_x_orig = np.array(train_dataset["train_set_x"][:])
    #extract the training labels
    train_set_y_orig = np.array(train_dataset["train_set_y"][:])
    #opening the test set .h5 file in read mode
    test_dataset = h5py.File("test_catvnoncat.h5", "r")
    #extracting the test images
    test_set_x_orig = np.array(test_dataset["test_set_x"][:])
    #extracting the test labels
    test_set_y_orig = np.array(test_dataset["test_set_y"][:])
    #reshape labels to row vectors
    train_set_y_orig = train_set_y_orig.reshape(1, train_set_y_orig.shape[0])
    test_set_y_orig = test_set_y_orig.reshape(1, test_set_y_orig.shape[0])
    #extracting class names
    classes = np.array(test_dataset["list_classes"][:])
    #return all components of the dataset
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes
#load the original train/test dataset using the function above
train_x_orig, train_y_orig, test_x_orig, test_y_orig, classes = load_dataset()
#number of examples in original training and test sets
m_train = train_x_orig.shape[0]
m_test = test_x_orig.shape[0]
num_px = train_x_orig.shape[1]
#flatten images and normalize pixel values to [0, 1]
train_x = train_x_orig.reshape(m_train, -1).T / 255.
test_x = test_x_orig.reshape(m_test, -1).T / 255.
train_y = train_y_orig
test_y = test_y_orig
#sigmoid activation function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))
#intialize parameter w and b 
def initialize_parameters(dim):
    w = np.zeros((dim, 1))
    b = 0.0
    return w, b
#forward and backward propogation
def propagate(w, b, X, Y):
    m = X.shape[1]
    A = sigmoid(np.dot(w.T, X) + b)
    cost = -np.sum(Y * np.log(A + 1e-15) + (1 - Y) * np.log(1 - A + 1e-15)) / m
    dw = np.dot(X, (A - Y).T) / m
    db = np.sum(A - Y) / m
    return dw, db, cost
#gradient descent optimization loop
def optimize(w, b, X, Y, num_iterations, learning_rate):
    for i in range(num_iterations):
        dw, db, cost = propagate(w, b, X, Y)
        w = w - learning_rate * dw
        b = b - learning_rate * db
    return w, b
#predict labels 
def predict(w, b, X):
    A = sigmoid(np.dot(w.T, X) + b)
    Y_prediction = (A > 0.5).astype(int)
    return Y_prediction
#dimensions of feature vector
dim = train_x.shape[0]
#initialize and train logistic regression model
w, b = initialize_parameters(dim)
w, b = optimize(w, b, train_x, train_y, num_iterations=2000, learning_rate=0.01)
#predictions on new training and new test sets
train_pred = predict(w, b, train_x)
test_pred = predict(w, b, test_x)
#Computing training and test accuracy 
train_accuracy = 100 - np.mean(np.abs(train_pred - train_y)) * 100
test_accuracy = 100 - np.mean(np.abs(test_pred - test_y)) * 100

print("Training accuracy:", train_accuracy)
print("Test accuracy:", test_accuracy)
print("w shape:", w.shape, "b:", b)
#finding indicesof misclassified
misclassified_indices = np.where(test_pred != test_y)[1]
print("Misclassified test indices:", misclassified_indices)
#Plotting up to 4 misclassified test images
n_show = min(4, misclassified_indices.shape[0])
plt.figure(figsize=(8, 8))
for i in range(n_show):
    idx = misclassified_indices[i]
    image = test_x_orig[idx]
    label_true = test_y[0, idx]
    label_pred = test_pred[0, idx]
    plt.subplot(2, 2, i + 1)
    plt.imshow(image)
    plt.axis("off")
    plt.title(f"idx {idx} | true {label_true} | pred {label_pred}")
plt.tight_layout()
plt.show()


Training accuracy: 99.52153110047847
Test accuracy: 70.0
w shape: (12288, 1) b: -0.00383066542394885
Misclassified test indices: [ 5  6 10 11 13 18 19 28 29 30 34 41 44 45 46]
